In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
from torchtext import data
from transformers import T5Tokenizer, T5Model

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [3]:
tokens = tokenizer.tokenize('Hello world how are you?')

print(tokens)

['▁Hello', '▁world', '▁how', '▁are', '▁you', '?']


In [4]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[8774, 296, 149, 33, 25, 58]


In [5]:
init_token = tokenizer.pad_token
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

<pad> </s> <pad> <unk>


In [6]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

0 1 0 2


In [7]:
max_input_length = tokenizer.max_model_input_sizes['t5-small']

print(max_input_length)

512


In [8]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [9]:
SRC = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

TRG = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

In [10]:
fields = [('src', SRC), ('trg', TRG)]

In [11]:
train_data = data.TabularDataset.splits(
                path = '',
                train = 'squad.csv',
                format = 'csv',
                fields = fields,
                skip_header = True)

train_data , valid_data = train_data[0].split(split_ratio=0.96,
                                             random_state = random.seed(12))

In [12]:
print(len(train_data.examples))
print(len(valid_data.examples))

49003
2042


In [13]:
print(vars(train_data.examples[10000]))

{'src': [2625, 3, 10, 788, 12, 8, 5919, 13, 1668, 11, 165, 20187, 16595, 6, 3, 11390, 208, 1632, 8, 167, 359, 1668, 2050, 11, 5752, 859, 8, 9026, 7, 117, 2459, 8, 2488, 13, 3, 11390, 208, 6886, 1827, 96, 15247, 121, 147, 8, 3825, 844, 5, 48, 22277, 5012, 13999, 45, 3, 9, 17952, 13, 1435, 16246, 7064, 447, 384, 9026, 7, 1928, 15, 26, 544, 16, 46, 1941, 12, 993, 8, 5383, 10312, 11, 4405, 8, 7596, 13, 8, 1322, 5, 48, 7021, 1597, 8, 166, 779, 3119, 16, 8, 3, 4502, 31, 11, 47, 8, 166, 14538, 607, 13, 1044, 18, 27818, 5, 822, 3, 10, 125, 686, 13, 789, 764, 45, 8, 17952, 13, 1435, 16246, 11015, 75, 384, 58], 'trg': [166, 14538, 607, 13, 1044, 18, 27818]}


In [14]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[10000])['src'])

print(tokens)
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[10000])['trg'])

print(tokens)

['▁context', '▁', ':', '▁due', '▁to', '▁the', '▁expansion', '▁of', '▁trade', '▁and', '▁its', '▁geographical', '▁proximity', ',', '▁', 'kie', 'v', '▁became', '▁the', '▁most', '▁important', '▁trade', '▁centre', '▁and', '▁chief', '▁among', '▁the', '▁commune', 's', ';', '▁therefore', '▁the', '▁leader', '▁of', '▁', 'kie', 'v', '▁gained', '▁political', '▁"', 'control', '"', '▁over', '▁the', '▁surrounding', '▁areas', '.', '▁this', '▁prince', 'dom', '▁emerged', '▁from', '▁', 'a', '▁coalition', '▁of', '▁traditional', '▁patri', 'arch', 'ic', '▁family', '▁commune', 's', '▁band', 'e', 'd', '▁together', '▁in', '▁an', '▁effort', '▁to', '▁increase', '▁the', '▁applicable', '▁workforce', '▁and', '▁expand', '▁the', '▁productivity', '▁of', '▁the', '▁land', '.', '▁this', '▁union', '▁developed', '▁the', '▁first', '▁major', '▁cities', '▁in', '▁the', '▁', 'rus', "'", '▁and', '▁was', '▁the', '▁first', '▁notable', '▁form', '▁of', '▁self', '-', 'government', '.', '▁question', '▁', ':', '▁what', '▁type', '▁of', 

In [15]:
class T5Network(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        self.t5 = t5 = T5Model.from_pretrained('t5-small')
        
        self.out = nn.Linear(t5.config.to_dict()['d_model'], t5.config.to_dict()['vocab_size'])
                
    def forward(self, src, trg):
        
        embedded = self.t5(input_ids=src, decoder_input_ids=trg)
        
        output = self.out(embedded[0])
        
        return output

In [16]:
model = T5Network().cuda()
model.load_state_dict(torch.load('model_4.pt'))

Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 76,988,544 trainable parameters


In [53]:
def translate_sentence(sentence, src_field, trg_field, model, max_len = 50):
    model.eval()

    src_indexes = [init_token_idx] + sentence + [eos_token_idx]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).cuda()

    trg_indexes = [init_token_idx]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).cuda()
        
        with torch.no_grad():
            output = model(src_tensor, trg_tensor)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == eos_token_idx:
            break
            
    return trg_indexes[1:-1]

In [54]:
src = vars(valid_data.examples[10])['src']
trg = vars(valid_data.examples[10])['trg']
translation = translate_sentence(src, SRC, TRG, model)

print(f"SRC : {' '.join(tokenizer.convert_ids_to_tokens(src))}")
print(f"TRG : {' '.join(tokenizer.convert_ids_to_tokens(trg))}")
print(f"PREDICTED : {' '.join(tokenizer.convert_ids_to_tokens(translation))}\n")

SRC : ▁context ▁ : ▁during ▁the ▁last ▁three ▁decades ▁of ▁the ▁19 th ▁century ▁ a ▁proliferation ▁of ▁ anthropo logical ▁societies ▁and ▁associations ▁occurred , ▁most ▁independent , ▁most ▁publishing ▁their ▁own ▁journals , ▁and ▁all ▁international ▁in ▁membership ▁and ▁association . ▁question ▁ : ▁what ▁pro life rated ▁in ▁the ▁last ▁three ▁decades ▁of ▁the ▁19 th ▁century ?
TRG : ▁ anthropo logical ▁societies
PREDICTED : ▁ anthropo logical ▁societies ▁and ▁associations



In [55]:
print(src)

[2625, 3, 10, 383, 8, 336, 386, 4160, 13, 8, 957, 189, 2646, 3, 9, 27419, 13, 3, 21468, 6207, 18804, 11, 10906, 6935, 6, 167, 2547, 6, 167, 9002, 70, 293, 18178, 6, 11, 66, 1038, 16, 4757, 11, 6028, 5, 822, 3, 10, 125, 813, 4597, 4094, 16, 8, 336, 386, 4160, 13, 8, 957, 189, 2646, 58]


In [56]:
print(tokenizer.convert_ids_to_tokens(src))

['▁context', '▁', ':', '▁during', '▁the', '▁last', '▁three', '▁decades', '▁of', '▁the', '▁19', 'th', '▁century', '▁', 'a', '▁proliferation', '▁of', '▁', 'anthropo', 'logical', '▁societies', '▁and', '▁associations', '▁occurred', ',', '▁most', '▁independent', ',', '▁most', '▁publishing', '▁their', '▁own', '▁journals', ',', '▁and', '▁all', '▁international', '▁in', '▁membership', '▁and', '▁association', '.', '▁question', '▁', ':', '▁what', '▁pro', 'life', 'rated', '▁in', '▁the', '▁last', '▁three', '▁decades', '▁of', '▁the', '▁19', 'th', '▁century', '?']


In [57]:
CONTEXT = "harry is playing with his dog. the dog is twenty years old."
QUERY = "how old is the dog ?"

text = "context : " + CONTEXT.lower() + " question : " + QUERY.lower()
tokens = tokenizer.tokenize(text)

In [58]:
print(text)
print('\n')
print(tokens)
print('\n')
print(tokenizer.convert_tokens_to_ids(tokens))

context : harry is playing with his dog. the dog is twenty years old. question : how old is the dog ?


['▁context', '▁', ':', '▁', 'har', 'ry', '▁is', '▁playing', '▁with', '▁his', '▁dog', '.', '▁the', '▁dog', '▁is', '▁twenty', '▁years', '▁old', '.', '▁question', '▁', ':', '▁how', '▁old', '▁is', '▁the', '▁dog', '▁', '?']


[2625, 3, 10, 3, 3272, 651, 19, 1556, 28, 112, 1782, 5, 8, 1782, 19, 6786, 203, 625, 5, 822, 3, 10, 149, 625, 19, 8, 1782, 3, 58]


In [59]:
pred_tokens = translate_sentence(tokenizer.convert_tokens_to_ids(tokens), SRC, TRG, model)

In [60]:
print(pred_tokens)
print(' '.join(tokenizer.convert_ids_to_tokens(pred_tokens)))

[6786]
▁twenty
